In [ ]:
from wayne_utils import load_data, save_data
import os
from config import _ROOT_PATH   # _ALL_METHODS

from eval.SpCQL.EM_Metric import Neo4j_ExactMatch
from eval.SpCQL.CA_Metric import Neo4j_ComprehensionAccuracy
from eval.SpCQL.SA_Metric import Neo4j_SyntaxAccuracy
from eval.SpCQL.EA_Metric import Neo4j_ExecutionAccuracy_Match
from eval.SpCQL.R3_EA_Metric import Neo4j_ExecutionAccuracy_R3

from utils import Record_result, Check_result

/home/jiangpeiwen2/jiangpeiwen2/miniconda3/envs/t2c/lib/python3.13/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [4]:
_REMOVE_LIMIT_SKIP = False

def selet_no_limit( predict_query_list, predict_result_list, gold_query_list, gold_result_list ):
    new_predict_query_list, new_predict_result_list, new_gold_query_list, new_gold_result_list = [], [], [], []
    for i in range(len(gold_query_list)):
        gold_query = gold_query_list[i]
        if "limit " not in gold_query.lower() and "skip " not in gold_query.lower():
            new_predict_query_list.append( predict_query_list[i])
            new_predict_result_list.append( predict_result_list[i])
            new_gold_query_list.append( gold_query_list[i])
            new_gold_result_list.append( gold_result_list[i])

    return new_predict_query_list, new_predict_result_list, new_gold_query_list, new_gold_result_list

In [5]:
import os
os.environ["CUDA_VISIBLE_DEVICES"] = "5"

In [ ]:
gold_query_list = load_data( os.path.join( _ROOT_PATH, "data/SpCQL/SpCQL_gold_query_list.json" ), "json")
gold_result_list = load_data( os.path.join( _ROOT_PATH, "data/SpCQL/SpCQL_gold_result_list.json"), "json")

allow_list = {
    "EM": True,
    "SA": True,
    "CA": True,
    "R3_EA": True,
    "EA": True
}
for method in ["FT"]: #_ALL_METHODS.keys():
    print( method )
    for sub_method in  ["GLM4-9B"]:     # _ALL_METHODS[method].keys(): 
        # if methods[method][sub_method]:
        
        whole_method_name = method +"-" + sub_method if sub_method != "" else method
        sub_method = None if sub_method == "" else sub_method
        print( whole_method_name )
        
        if method == "Gold":
            predict_query_path = os.path.join( _ROOT_PATH, "data/SpCQL/SpCQL_gold_query_list.json" )
            predict_result_path = os.path.join( _ROOT_PATH, "data/SpCQL/SpCQL_gold_query_predict_result_list.json" )
        else:
            predict_query_path = os.path.join( _ROOT_PATH, f"inter_data/SpCQL/query_list/{whole_method_name}_query_list.json" )
            predict_result_path = os.path.join( _ROOT_PATH, f"inter_data/SpCQL/execute_result_list/{whole_method_name}_execute_results.json")
            
        if os.path.exists( predict_query_path ) and os.path.exists( predict_result_path ):
            print( f"{whole_method_name}的query list和exec result都存在，可以进行eval")
            
            predict_query_list = load_data( predict_query_path, "json")
            predict_result_list = load_data( predict_result_path, "json")

            if _REMOVE_LIMIT_SKIP:
                predict_query_list, predict_result_list, gold_query_list, gold_result_list = selet_no_limit( predict_query_list, predict_result_list, gold_query_list, gold_result_list )
            if allow_list[ "EM" ] and Check_result( "EM", method, dataset = "SpCQL", sub_method = sub_method, overwrite = False):
                EM, EM_list, _, _ = Neo4j_ExactMatch(gold_query_list,  predict_query_list)          # query匹配
                Record_result( "EM", EM, method, "SpCQL", sub_method)

            if allow_list[ "CA" ] and Check_result( "CA", method, dataset = "SpCQL", sub_method = sub_method, overwrite = False):
                CA = Neo4j_ComprehensionAccuracy( gold_query_list,  predict_query_list )   # query语义相似度
                Record_result( "CA", CA, method, "SpCQL", sub_method)
            
            if allow_list["SA"] or allow_list["R3_EA"] or allow_list["EA"]:
                SA, ER, SA_list = Neo4j_SyntaxAccuracy( predict_result_list )                       # query语义错误
            if allow_list[ "SA" ] and Check_result( "SA", method, dataset = "SpCQL", sub_method = sub_method, overwrite = False):
                Record_result( "SA", SA, method, "SpCQL", sub_method)
                Record_result( "ER", ER, method, "SpCQL", sub_method)

            if allow_list[ "R3_EA" ] and Check_result( "R3_EA", method, dataset = "SpCQL", sub_method = sub_method, overwrite = False):
                R3_EA, R3_IEA = Neo4j_ExecutionAccuracy_R3( predict_result_list, gold_result_list, SA_list )
                Record_result( "R3_EA", R3_EA, method, "SpCQL", sub_method)
                Record_result( "R3_IEA", R3_IEA, method, "SpCQL", sub_method)
            
            if allow_list[ "EA" ] and Check_result( "EA", method,  dataset = "SpCQL", sub_method = sub_method, overwrite = False):
                EA, EEA, IEA, EA_list, EEA_list = Neo4j_ExecutionAccuracy_Match( predict_result_list, gold_result_list, SA_list)     # 结果精确
                Record_result( "EA", EA, method, "SpCQL", sub_method)
                Record_result( "EEA", EEA, method, "SpCQL", sub_method)
                Record_result( "IEA", IEA, method, "SpCQL", sub_method)
        else:
            print( f"路径不存在{predict_query_path}或者{predict_result_path}")
            """"""

FT
FT-GLM4-9B
FT-GLM4-9B的query list和exec result都存在，可以进行eval
不存在记录FT-GLM4-9B-EM，允许重写。
新的记录：FTGLM4-9B的EM = 53.81165919282511
不存在记录FT-GLM4-9B-CA，允许重写。


Some weights of RobertaModel were not initialized from the model checkpoint at roberta-large and are newly initialized: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
Bert Scoring: 237it [00:04, 87.54it/s]Warning: Empty reference sentence detected; setting raw BERTScores to 0.
Bert Scoring: 275it [00:05, 89.36it/s]Warning: Empty reference sentence detected; setting raw BERTScores to 0.
Bert Scoring: 315it [00:05, 89.63it/s]Warning: Empty reference sentence detected; setting raw BERTScores to 0.
Bert Scoring: 383it [00:06, 90.31it/s]Warning: Empty reference sentence detected; setting raw BERTScores to 0.
Bert Scoring: 393it [00:06, 90.91it/s]Warning: Empty reference sentence detected; setting raw BERTScores to 0.
Bert Scoring: 482it [00:07, 91.33it/s]Warning: Empty reference sentence detected; setting raw BERTScores to 0.
Bert Scoring: 600it [00:09, 85.16it/s]Warn

新的记录：FTGLM4-9B的CA = 88.20250054155943
不存在记录FT-GLM4-9B-SA，允许重写。
新的记录：FTGLM4-9B的SA = 96.01395117090185
新的记录：FTGLM4-9B的ER = 4.384653712007972
不存在记录FT-GLM4-9B-N3_EA，允许重写。


N3 EA evaluating...:   0%|          | 0/2007 [00:00<?, ?it/s]Building prefix dict from the default dictionary ...
Loading model from cache /tmp/jieba.cache
Loading model cost 0.650 seconds.
Prefix dict has been built successfully.
N3 EA evaluating...: 100%|██████████| 2007/2007 [01:21<00:00, 24.54it/s] 


新的记录：FTGLM4-9B的N3_EA = 76.9930012466757
新的记录：FTGLM4-9B的N3_IEA = 80.18938946656883
不存在记录FT-GLM4-9B-EA，允许重写。
新的记录：FTGLM4-9B的EA = 49.7259591429995
新的记录：FTGLM4-9B的EEA = 55.30642750373692
新的记录：FTGLM4-9B的IEA = 51.79034769071095
